### Ant Colony Optimization on Neural Networks
#### References

- Dorigo, Marco & Birattari, Mauro & Stützle, Thomas. (2006). Ant Colony Optimization. Computational Intelligence Magazine, IEEE. 1. 28-39. 10.1109/MCI.2006.329691. 
- Socha, K., Blum, C. An ant colony optimization algorithm for continuous optimization: application to feed-forward neural network training. Neural Comput & Applic 16, 235–247 (2007). https://doi.org/10.1007/s00521-007-0084-z (Not implemented)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,classification_report
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
data = pd.read_csv("data.csv")
print(data.head())

df = data
df['Personal Loan'] = data['Personal Loan']


X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , y , test_size=0.2, random_state=0)

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def derivative_sig(x):
    return x * (1 - x)


def forward(
    X, y,W1,W2, input_layer, hidden1, output_layer, iter_, learning_rate
):
    # weights =[]
    # act =[]


    # weights=[W1,W2]

    for i in range(iter_):

        z1 = np.dot(X, W1)
        a1 = sigmoid(z1)
        z2 = np.dot(a1, W2)
        y_pred = np.int32(sigmoid(z2))

        error = classification_report(y, y_pred)
        delta3 = (y_pred - y) * derivative_sig(y_pred)
        dW2 = np.dot(a1.T, delta3)
        delta2 = np.dot(delta3, W2.T) * derivative_sig(a1)
        dW1 = np.dot(X.T, delta2)

        W1 -= learning_rate * dW1
        W2 -= learning_rate * dW2

    print(f"Classification Report after {iter_} : {error}")

    return W1, W2


def ACO(X, y, W1, W2, iter_, antcount, decay, var1, var2, var3):

    weights_ = np.zeros(W1.size + W2.size)
    error_ = np.inf
    pheromones = np.ones((W1.size + W2.size, 2))

    for i in range(iter_):

        for j in range(antcount):

            ant_weights = np.zeros(W1.size + W2.size)

            for k in range(len(ant_weights)):

                if np.random.rand() < pheromones[k][1]:
                    ant_weights[k] = np.random.uniform(-1, 1)
                else:
                    ant_weights[k] = weights_[k]

            W1 = ant_weights[: W1.size].reshape(W1.shape)
            W2 = ant_weights[W1.size :].reshape(W2.shape)

            y_pred = sigmoid(np.dot(sigmoid(np.dot(X, W1)), W2))
            error = np.mean(np.square(y - y_pred))

            if error < error_:
                weights_ = ant_weights
                weights_ = ant_weights.copy()
                error_ = error

            delta3 = (y_pred - y) * derivative_sig(y_pred)
            dW2 = np.dot(sigmoid(np.dot(X, W1)).T, delta3)

            delta2 = np.dot(delta3, W2.T) * derivative_sig(sigmoid(np.dot(X, W1)))
            dW1 = np.dot(X.T, delta2)

            trail_update = np.zeros((W1.size + W2.size, 2))

            # for k in range(len(trail_update/2)):
            #     if ant_weights == weights_:

            for k in range(len(trail_update)):

                if ant_weights[k] == weights_[k]:

                    trail_update[k][0] = (1 - decay) * pheromones[k][0] + var3 / error_
                    trail_update[k][1] = var1 / (var1 + var2)


                else:

                    trail_update[k][0] = (1 - decay) * pheromones[k][0]
                    trail_update[k][1] = var2 / (var1 + var2)
                    
            pheromones = trail_update

            W1 -= var1 * dW1
            W2 -= var1 * dW2
    return W1, W2

In [20]:
input_layer = X.shape[1]
hidden1 = 4
output_layer = 1
iter_ = 1000
learning_rate = 0.1
antcount = 10
decay = 0.5
var1 = 0.1
var2 = 0.9
var3 = 1

W1 = np.random.randn(input_layer, hidden1)
W2 = np.random.randn(hidden1, output_layer)
w1,w2 =forward(x_train,y_train,W1,W2,input_layer,hidden1,output_layer,iter_,learning_rate)
# print(f"\nInitial Population weight :\n{w1,w2}\n")
# print(f"After ACO :\n{ACO(X,y,w1,w2,iter_,antcount,decay,var1,var2,var3)}")
W1,W2 = ACO(x_train,y_train,w1,w2,iter_,antcount,decay,var1,var2,var3)
w1,w2 = forward(x_test,y_test,W1,W2,input_layer,hidden1,output_layer,iter_,learning_rate)

Classification Report after 1000 :

               precision    recall  f1-score   support

           0       0.40      1.00      0.57         2
           1       1.00      0.67      0.80         9

    accuracy                           0.73        11
   macro avg       0.70      0.83      0.69        11
weighted avg       0.89      0.73      0.76        11

